In [1]:
'''
Código para obtener las series temporales de las variables de interes.
En este caso:
    - t2m
'''

import xarray as xr
import pandas as pd
import glob


path= '/home/cony/Documentos/Cony/trabajo_final/para_subir/ERA5_Data_Salta/'
files = sorted(glob.glob(f"{path}*.nc"))


# Punto de interés
lat_sla = -24.7
lon_sla = -65.4

# Listas para guardar los valores
temperature_series = []
dewpoint_temperature_series = []

# Procesar cada archivo
for f in files:
    ds = xr.open_dataset(f)
    
    # Mostrar las coordenadas disponibles (solo la primera vez)
    if not temperature_series:
        print("Latitudes disponibles:", ds.latitude.values)
        print("Longitudes disponibles:", ds.longitude.values)
    
    # Seleccionar el punto más cercano
    try:
        if 't2m' in ds.data_vars:
            temperature = ds['t2m'].sel(latitude=lat_sla, longitude=lon_sla, method='nearest')
            temperature_series.append(temperature.to_dataframe().reset_index())
    except KeyError as e:
        print(f"Error al seleccionar datos en el archivo {f}: {e}")
    finally:
        ds.close()

# Concatenar todas las series
if temperature_series:
    df_temperature = pd.concat(temperature_series, ignore_index=True)
    df_temperature['t2m'] = df_temperature['t2m'] - 273.15  # Convertir a °C si está en Kelvin
    df_temperature = df_temperature[['valid_time', 'number', 'latitude', 'longitude', 't2m']].copy()
    df_temperature.to_csv('temperature_series.csv', index=False)
    print("Datos de temperatura guardados en 'temperature_series.csv'.")


if not temperature_series:
    print("No se pudieron extraer datos válidos.")

Latitudes disponibles: [-24.75]
Longitudes disponibles: [-65.5]
Datos de temperatura guardados en 'temperature_series.csv'.


In [2]:
df_temperature.head(2), df_temperature.tail(2), df_temperature.info(), df_temperature.t2m.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17544 entries, 0 to 17543
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   valid_time  17544 non-null  datetime64[ns]
 1   number      17544 non-null  int64         
 2   latitude    17544 non-null  float64       
 3   longitude   17544 non-null  float64       
 4   t2m         17544 non-null  float32       
dtypes: datetime64[ns](1), float32(1), float64(2), int64(1)
memory usage: 616.9 KB


(           valid_time  number  latitude  longitude        t2m
 0 2023-01-01 00:00:00       0    -24.75      -65.5  20.844391
 1 2023-01-01 01:00:00       0    -24.75      -65.5  20.472321,
                valid_time  number  latitude  longitude        t2m
 17542 2024-12-31 22:00:00       0    -24.75      -65.5  22.305817
 17543 2024-12-31 23:00:00       0    -24.75      -65.5  21.170807,
 None,
 count    17544.000000
 mean        14.936983
 std          5.937288
 min         -4.646576
 25%         10.829681
 50%         15.460602
 75%         19.002899
 max         31.347076
 Name: t2m, dtype: float64)

In [3]:
import matplotlib.pyplot as plt
%matplotlib qt

plt.plot(df_temperature['valid_time'], df_temperature['t2m'], label='Temperatura (°C)', linestyle='--')
plt.legend()
plt.grid()
plt.show()


In [4]:
df_temperature.to_csv('/home/cony/Documentos/Cony/trabajo_final/para_subir/datos/datos_era_sla.csv', index=False)